In [33]:
import sys
import time

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
# import seaborn as sns
# sns.set()
from sklearn import tree
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import seaborn as sns
from fuzzytree import FuzzyDecisionTreeClassifier

In [34]:
filenames = {
    "emulab_hdd_merged": "emulab_d460_10Gbps_hdd_merged_V2.csv",
    "emulab_hdd_unmerged": "emulab_d460_10Gbps_hdd_unmerged_V2.csv",

    "utah_ssd_merged": "utah_c6525-25g_25Gbps_ssd_merged_V2.csv",
    "utah_ssd_unmerged": "utah_c6525-25g_25Gbps_ssd_unmerged_V2.csv",

    "wisconsin_ssd_merged": "wisconsin_c220g1-10Gbps_ssd_merged_V2.csv",
    "wisconsin_ssd_unmerged": "wisconsin_c220g1-10Gbps_ssd_unmerged_V2.csv",

    "utah_ssd_merged_1g": "utah_c6525-25g_1Gbps_ssd_merged_V2.csv",
    "utah_ssd_unmerged_1g": "utah_c6525-25g_1Gbps_ssd_unmerged_V2.csv",

    "wisconsin_hdd_ssd_merged": "wisconsin_c220g1-10Gbps_hdd_ssd_merged_V2.csv",
    "wisconsin_hdd-ssd_unmerged": "wisconsin_c220g1-10Gbps_hdd_ssd_unmerged_V2.csv",

    "utah_22_unmerged": "22_utah_c6525-25g_25Gbps_ssd_merged_V2.csv",
    "utah_22_merged": "22_utah_c6525-25g_25Gbps_ssd_unmerged_V2.csv"

}

In [35]:
utah_ssd_merged = pd.read_csv(filenames.get('utah_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
utah_ssd_merged_1g = pd.read_csv(filenames.get('utah_ssd_merged_1g')).drop(['time_stamp', 'through_put'], axis=1)
utah_22_merged = pd.read_csv(filenames.get('utah_22_merged')).drop(['time_stamp', 'through_put'], axis=1)
emulab_hdd_merged = pd.read_csv(filenames.get('emulab_hdd_merged')).drop(['time_stamp', 'through_put'], axis=1)
wisconsin_ssd_merged = pd.read_csv(filenames.get('wisconsin_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
wisconsin_ssd_unmerged = pd.read_csv(filenames.get('wisconsin_ssd_unmerged')).drop(['time_stamp', 'through_put'], axis=1)
wisconsin_hdd_ssd_merged = pd.read_csv(filenames.get('wisconsin_hdd_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
utah_ssd_merged_1g = pd.read_csv(filenames.get('utah_ssd_merged_1g')).drop(['time_stamp', 'through_put'], axis=1)

In [ ]:
# features = ['sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_retrans', 'sender_ssthresh_value', 'sender_segs_in', 'sender_avg_send_value', 'sender_tcp_snd_buffer_min', 'sender_tcp_snd_buffer_max', 'sender_req_active', 'sender_write_bytes', 'sender_avg_waittime_md', 'sender_system_cpu_percent', 'sender_remote_ost_write_bytes', 'receiver_seg_out', 'receiver_segs_in', 'receiver_req_waittime', 'receiver_req_active', 'receiver_write_bytes', 'receiver_ost_write', 'receiver_avg_waittime_md', 'receiver_system_cpu_percent', 'receiver_system_memory_percent', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']
#*
# features = ['sender_avg_rtt_value', 'sender_retrans','sender_segs_in', 'sender_seg_out', 'sender_req_waittime', 'sender_read_bytes', 'sender_write_bytes', 'sender_avg_waittime_md', 'sender_ost_read' , 'sender_ost_write','sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes','receiver_segs_in','receiver_seg_out', 'receiver_req_waittime', 'receiver_read_bytes', 'receiver_write_bytes', 'receiver_ost_read', 'receiver_ost_write', 'receiver_avg_waittime_md', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

# features = ['sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_retrans', 'sender_ssthresh_value', 'sender_segs_in', 'sender_req_active', 'sender_write_bytes', 'sender_remote_ost_write_bytes',
#             'receiver_seg_out', 'receiver_segs_in', 'receiver_req_waittime', 'receiver_req_active', 'receiver_write_bytes', 'receiver_ost_write' , 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

# features = ['sender_seg_out', "sender_retrans", 'sender_avg_waittime_md', 'receiver_segs_in', 'receiver_avg_waittime_md' ,'receiver_write_bytes',]

# features =['sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_seg_out', 'sender_ssthresh_value', 'sender_segs_in', 'sender_avg_send_value', 'sender_vsize', 'sender_rss', 'sender_mem_usage_percentage', 'sender_avg_waittime_md', 'sender_system_cpu_percent', 'receiver_seg_out', 'receiver_segs_in', 'receiver_read_bytes_io', 'receiver_vsize', 'receiver_rss', 'receiver_mem_usage_percentage', 'receiver_req_active', 'receiver_avg_waittime_md', 'receiver_system_cpu_percent']

# features = ['sender_avg_rtt_value', 'sender_pacing_rate', 'sender_cwnd_rate', 'sender_byte_ack', 'sender_seg_out', 'sender_retrans', 'sender_ssthresh_value', 'sender_segs_in', 'sender_avg_send_value', 'sender_unacked_value', 'sender_send_buffer_value', 'sender_cpu_usage_percentage', 'sender_tcp_snd_buffer_min', 'sender_tcp_snd_buffer_max', 'sender_req_waittime', 'sender_req_active', 'sender_read_bytes', 'sender_write_bytes', 'sender_ost_read', 'sender_ost_write', 'sender_pending_read_pages', 'sender_read_RPCs_in_flight', 'sender_avg_waittime_md', 'sender_system_cpu_percent', 'sender_system_memory_percent', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_avg_retransmission_timeout_value', 'receiver_seg_out', 'receiver_segs_in', 'receiver_wchar', 'receiver_syscw', 'receiver_read_bytes_io', 'receiver_cpu_usage_percentage', 'receiver_req_waittime', 'receiver_req_active', 'receiver_read_bytes', 'receiver_write_bytes', 'receiver_ost_read', 'receiver_ost_write', 'receiver_pending_read_pages', 'receiver_read_RPCs_in_flight', 'receiver_avg_waittime_md', 'receiver_req_waittime_md', 'receiver_system_cpu_percent', 'receiver_system_memory_percent', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

#19 features
# features = ['sender_seg_out', 'sender_segs_in', 'sender_avg_send_value', 'sender_send_buffer_value', 'sender_tcp_snd_buffer_min', 'sender_tcp_snd_buffer_max', 'sender_req_active', 'sender_read_bytes', 'sender_ost_read', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_seg_out', 'receiver_segs_in', 'receiver_read_bytes_io', 'receiver_req_active', 'receiver_write_bytes', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

# 14 features
# features = ['sender_seg_out', 'sender_avg_send_value', 'sender_tcp_snd_buffer_max', 'sender_read_bytes', 'sender_ost_read', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_seg_out', 'receiver_segs_in', 'receiver_read_bytes_io', 'receiver_write_bytes', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

# features = ['sender_seg_out', 'sender_avg_send_value', 'sender_tcp_snd_buffer_max', 'sender_read_bytes', 'sender_ost_read', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_write_bytes', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes']

# # #16 features
# features = ['sender_seg_out', 'sender_avg_send_value', 'sender_read_bytes_io', 'sender_write_bytes_io', 'sender_ost_read', 'sender_ost_write', 'sender_pending_read_pages', 'sender_read_RPCs_in_flight', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_segs_in', 'receiver_write_bytes_io', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes', 'sender_read_bytes', 'receiver_write_bytes'] #, 'sender_read_bytes', 'receiver_write_bytes', sender_tcp_snd_buffer_max

# features = ['sender_seg_out', 'sender_avg_send_value', 'sender_read_bytes_io', 'sender_write_bytes_io', 'sender_ost_read', 'sender_ost_write', 'sender_pending_read_pages', 'sender_read_RPCs_in_flight', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_segs_in', 'receiver_write_bytes_io', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes', 'sender_read_bytes', 'receiver_write_bytes']

# features = ['sender_seg_out', 'sender_avg_send_value', 'sender_read_bytes_io', 'sender_write_bytes_io', 'sender_ost_read', 'sender_ost_write', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_segs_in', 'receiver_write_bytes_io', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes', 'sender_read_bytes', 'receiver_write_bytes',
#                 'sender_pending_read_pages', 'sender_read_RPCs_in_flight',
#                 'sender_send_buffer_value', 'sender_req_active', 'receiver_req_active',
#                 #'sender_segs_in',
#                 'sender_retrans', 'sender_cwnd_rate']

# features = ['sender_seg_out', 'sender_avg_send_value', 'sender_read_bytes_io', 'sender_write_bytes_io', 'sender_ost_read', 'sender_ost_write', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_write_bytes_io', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes', 'sender_read_bytes', 'receiver_write_bytes',
#                 'receiver_segs_in',
#                 'sender_pending_read_pages', 'sender_read_RPCs_in_flight',
#                 'sender_send_buffer_value', 'sender_req_active', 'receiver_req_active',
#                 #'sender_segs_in',
#                 'sender_retrans', 'sender_cwnd_rate',
#                 'receiver_tcp_rcv_buffer_max',
#                 # 'sender_tcp_snd_buffer_max',
#                 #'sender_tcp_snd_buffer_min'
#                 ]

features = ['sender_seg_out', 'sender_avg_send_value', 'sender_read_bytes_io', 'sender_write_bytes_io', 'sender_ost_read', 'sender_ost_write', 'sender_remote_ost_read_bytes', 'sender_remote_ost_write_bytes', 'receiver_write_bytes_io', 'receiver_ost_write', 'receiver_remote_ost_read_bytes', 'receiver_remote_ost_write_bytes', 'sender_read_bytes', 'receiver_write_bytes',
                'receiver_segs_in',
                'sender_pending_read_pages', 'sender_read_RPCs_in_flight',
                'sender_send_buffer_value', 'sender_req_active', 'receiver_req_active',
                #'sender_segs_in',
                'sender_retrans', 'sender_cwnd_rate',
                'receiver_tcp_rcv_buffer_max',
                #------------
                #'sender_tcp_snd_buffer_max',
                #'sender_tcp_snd_buffer_min',
                'receiver_read_bytes_io',
                'receiver_ost_read',

                # 'sender_write_bytes',
                # 'receiver_read_bytes',
                ]

In [ ]:
#[ 'sender_tcp_snd_buffer_max','receiver_seg_out', 'receiver_read_bytes_io'] # 'sender_req_active', 'sender_segs_in', 'sender_send_buffer_value', 'sender_tcp_snd_buffer_min', 'receiver_req_active'
 ['sender_tcp_snd_buffer_max', 'sender_pending_read_pages', 'sender_read_RPCs_in_flight']

In [32]:
##[ 91, 100, 46]
d = {}
for i in range(109,161):
    d[i]=i-27
print(d)

{109: 82, 110: 83, 111: 84, 112: 85, 113: 86, 114: 87, 115: 88, 116: 89, 117: 90, 118: 91, 119: 92, 120: 93, 121: 94, 122: 95, 123: 96, 124: 97, 125: 98, 126: 99, 127: 100, 128: 101, 129: 102, 130: 103, 131: 104, 132: 105, 133: 106, 134: 107, 135: 108, 136: 109, 137: 110, 138: 111, 139: 112, 140: 113, 141: 114, 142: 115, 143: 116, 144: 117, 145: 118, 146: 119, 147: 120, 148: 121, 149: 122, 150: 123, 151: 124, 152: 125, 153: 126, 154: 127, 155: 128, 156: 129, 157: 130, 158: 131, 159: 132, 160: 133}


In [58]:
utah_ssd_unmerged = pd.read_csv(filenames.get('utah_ssd_unmerged'))
feature = 'sender_tcp_snd_buffer_max'
# feature = 'receiver_tcp_rcv_buffer_max'
# print(Counter(emulab_hdd_merged[feature]).keys())
print(Counter(utah_ssd_unmerged[feature]).keys())
# print(Counter(wisconsin_ssd_merged[feature]).keys())
# print(Counter(utah_ssd_merged_1g[feature]).keys())
# print(Counter(wisconsin_hdd_ssd_merged[feature]).keys())
# utah_ssd_merged_1g['sender_avg_rtt_value'] * ( 1 * 1000000)

dict_keys([4194304, 8192, 4096, 2048, 2097152, 524288, 1048576])


In [75]:
feature = 'sender_tcp_snd_buffer_max'
label = 150
print(utah_ssd_unmerged[utah_ssd_unmerged.label_value==label].through_put.mean())
utah_ssd_unmerged[utah_ssd_unmerged.label_value==label][['sender_tcp_snd_buffer_max', 'through_put']]

3.320449953550821


,sender_tcp_snd_buffer_max,through_put
24860,2048,1.270772
24861,2048,2.350014
24862,2048,1.977701
24863,2048,2.882284
24864,2048,3.935919
...,...,...
25037,2048,4.084983
25038,2048,4.059717
25039,2048,3.813379
25040,2048,4.019482


In [51]:
# emulab_hdd_merged.drop(columns=['label_value'])
# emulab_hdd_merged.sender_seg_out
emulab_hdd_merged.loc[emulab_hdd_merged.sender_seg_out==0, 'sender_seg_out']= 1
emulab_hdd_merged.sender_seg_out
emulab_hdd_merged[emulab_hdd_merged.label_value ==0]['sender_avg_rtt_value'].mean()

#np.round(emulab_hdd_merged[emulab_hdd_merged.label_value ==0]['sender_avg_rtt_value'].mean(), 2)
#10 * (10 ** 9) * 0.84 * (10** -3) / 8

0.8386956521739131

In [ ]:
# 142
emulab_hdd_merged.receiver_tcp_rcv_buffer_max
emulab_hdd_merged.groupby('label_value').sender_tcp_snd_buffer_max.mean()
# emulab_hdd_merged.receiver_send_buffer_value
# emulab_hdd_merged["sender_segs_in"] = emulab_hdd_merged.sender_segs_in / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].receiver_seg_out.mean()
# emulab_hdd_merged.sender_segs_in.mean()
# emulab_hdd_merged["sender_retrans"] = emulab_hdd_merged.sender_retrans / emulab_hdd_merged.sender_seg_out
# emulab_hdd_merged.sender_retrans.mean()
# emulab_hdd_merged["sender_send_buffer_value"] = emulab_hdd_merged["sender_send_buffer_value"] / emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_send_buffer_value.mean()
# emulab_hdd_merged[emulab_hdd_merged.label_value == 0].sender_ost_read
# print(emulab_hdd_merged[['sender_ssthresh_value','sender_cwnd_rate']])
# print(emulab_hdd_merged.sender_ssthresh_value / emulab_hdd_merged.sender_cwnd_rate)

In [ ]:
# wisconsin_ssd_merged.receiver_write_bytes.mean()/410
wisconsin_ssd_merged["sender_segs_in"] = wisconsin_ssd_merged.sender_segs_in / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].receiver_seg_out.mean()
wisconsin_ssd_merged.sender_segs_in.mean()
# wisconsin_ssd_merged["sender_retrans"] = wisconsin_ssd_merged["sender_retrans"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_retrans.mean()
# wisconsin_ssd_merged["sender_retrans"] = wisconsin_ssd_merged.sender_retrans / wisconsin_ssd_merged.sender_seg_out
# utah_ssd_merged_1g.groupby('label_value').sender_retrans.mean()
# wisconsin_ssd_merged.sender_retrans.mean()


# wisconsin_ssd_merged["sender_send_buffer_value"] = wisconsin_ssd_merged["sender_send_buffer_value"] / wisconsin_ssd_merged[wisconsin_ssd_merged.label_value == 0].sender_send_buffer_value.mean()

In [ ]:
# df1 =
df1 = wisconsin_ssd_merged.copy()
df2 = emulab_hdd_merged.copy()
feature_name = 'sender_seg_out'

# print(df1.receiver_remote_ost_read_bytes.describe().mean)
# print(df2.receiver_remote_ost_read_bytes.describe())
print(df1[feature_name].mean()/df2[feature_name].mean())

df1 = normalize_df(df1)
df2 = normalize_df(df2)

# print(df1.receiver_remote_ost_read_bytes.describe().mean)
# print(df2.receiver_remote_ost_read_bytes.describe())
print(df1[feature_name].mean()/df2[feature_name].mean())


In [ ]:
def normalize_df(df):
    df['sender_retrans'] = df['sender_retrans'] / df[df.label_value == 0].sender_seg_out.mean()
    # TODO: CORRECT THIS
    # df["sender_remote_ost_write_bytes"] = df["sender_remote_ost_write_bytes"] / df[df.label_value == 0].receiver_remote_ost_write_bytes.mean()
    # TODO: CORRECT THIS
    # df["receiver_remote_ost_read_bytes"] = df["receiver_remote_ost_read_bytes"] / df[df.label_value == 0].sender_remote_ost_read_bytes.mean()
    # TODO CORRECT THIS
    # df["sender_ost_write"] = df["sender_ost_write"] / df[df.label_value == 0].receiver_ost_write.mean()


    df["sender_seg_out"] = df["sender_seg_out"] / df[df.label_value == 0].sender_seg_out.mean()
    # df["sender_avg_send_value"] = df["sender_avg_send_value"] / df[df.label_value == 0].sender_avg_send_value.mean()
    # df["sender_ost_read"] = df["sender_ost_read"] / df[df.label_value == 0].sender_ost_read.mean()
    df["sender_read_bytes"] = df["sender_read_bytes"] / df[df.label_value == 0].sender_read_bytes.mean()
    df["sender_remote_ost_read_bytes"] =df["sender_remote_ost_read_bytes"] / df[df.label_value == 0].sender_remote_ost_read_bytes.mean()
    df["receiver_segs_in"] = df["receiver_segs_in"] / df[df.label_value == 0].receiver_segs_in.mean()
    df["receiver_write_bytes"] = df["receiver_write_bytes"] / df[df.label_value == 0].receiver_write_bytes.mean()
    # df["receiver_ost_write"] = df["receiver_ost_write"] / df[df.label_value == 0].receiver_ost_write.mean()
    df["receiver_remote_ost_write_bytes"] = df["receiver_remote_ost_write_bytes"] / df[df.label_value == 0].receiver_remote_ost_write_bytes.mean()

    # df["sender_write_bytes_io"] = df["sender_write_bytes_io"] / df[df.label_value == 0].sender_write_bytes_io.mean()
    # df["sender_read_bytes_io"] = df["sender_read_bytes_io"] / df[df.label_value == 0].sender_read_bytes_io.mean()
    # df["receiver_write_bytes_io"] = df["receiver_write_bytes_io"] / df[df.label_value == 0].receiver_write_bytes_io.mean()

    # df["sender_ssthresh_value"] = df.sender_ssthresh_value / df.sender_cwnd_rate
    # df['sender_retrans'] = df['sender_retrans'] / df[df.label_value == 0].sender_seg_out.mean()

    return df



In [ ]:
def remove_labels_in_df(df, labels_values):
    for lbl in labels_values:
        df = df.drop(df[df.label_value == lbl].index)
    return df

In [ ]:
# df = emulab_hdd_merged.copy()
# df = wisconsin_ssd_merged.copy()
# df = wisconsin_ssd_unmerged.copy()
df = utah_ssd_merged_1g.copy()
remove_labels = [ 91, 100, 46] #[]#[ 91, 100, 46]
df = remove_labels_in_df(df, remove_labels)
df = normalize_df(df)
df = grouped_to_5_cat(df,161)
# # df = utah_ssd_merged.copy()
# df = wisconsin_ssd_unmerged.copy()
# print(df.groupby('label_value').size())
# print(df.shape)
# df.loc[df.label_value != 0, "label_value"] = 1
test_df = df.groupby('label_value').sample(0)
# test_df = df.loc[df['label_value']==0].sample(75)
# print(test_df.groupby('label_value').size())
# print(test_df.shape)
# test_df

# train_df = df.drop(test_df.index)

train_df = df
# print(train_df.shape)

y = train_df.label_value
X = train_df[features] #.drop(columns=["label_value", "receiver_avg_waittime_md", "sender_avg_waittime_md", "sender_req_waittime", "sender_system_cpu_percent", "sender_cpu_usage_percentage", "sender_system_memory_percent","receiver_cpu_usage_percentage", "receiver_req_waittime", "receiver_system_cpu_percent", "receiver_system_memory_percent",'sender_avg_rtt_value', 'sender_cwnd_rate', 'sender_byte_ack', 'sender_retrans', 'sender_ssthresh_value','sender_pacing_rate', 'sender_avg_retransmission_timeout_value', 'sender_unacked_value', 'receiver_syscw', 'receiver_req_waittime_md', 'receiver_avg_retransmission_timeout_value', 'sender_segs_in', 'sender_tcp_snd_buffer_min','receiver_seg_out', 'receiver_read_bytes_io', 'sender_send_buffer_value','sender_req_active','sender_mem_usage_percentage','sender_req_waittime_md', 'sender_mds_close_md', 'receiver_req_active', 'receiver_mds_close_md', 'sender_rchar', 'sender_syscr', 'receiver_rchar', 'receiver_wchar', 'receiver_mds_getattr_lock_md', 'receiver_ldlm_cancel_md', 'sender_vsize', 'receiver_syscr', 'sender_rss',  'receiver_vsize', 'receiver_rss', 'sender_mds_getattr_lock_md', 'receiver_mem_usage_percentage', 'sender_read_bytes', 'sender_write_bytes', 'receiver_write_bytes'])#.drop(columns="label_value")
X_train, X_test, y_train, y_test = train_test_split(X,y)
print(X_train.shape)
# X_train, y_train = RandomUnderSampler(sampling_strategy="all").fit_resample(X_train, y_train)
X_train, y_train = RandomOverSampler(sampling_strategy="all").fit_resample(X_train, y_train)

print(X_train.shape)
# clf = RandomForestClassifier(n_estimators=100)
# clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100))
# clf = make_pipeline(StandardScaler(), tree.DecisionTreeClassifier())
# clf = make_pipeline(StandardScaler(), FuzzyDecisionTreeClassifier())
clf = tree.DecisionTreeClassifier(max_depth=9)
# clf = FuzzyDecisionTreeClassifier()
# clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))

# y_pred
# y_pred_5cat = change_labels_5(y_pred, 21) #change_labels_21(y_pred, 161)
# y_test_5cat = change_labels_5(y_test, 21) #change_labels_21(y_test, 161)
# print(np.round(metrics.accuracy_score(y_test_5cat, y_pred_5cat) * 100, 2))
# y_pred_5cat[y_pred_5cat[:]==55]
# y_pred_5cat
# Counter(y_pred_5cat)

In [ ]:
score = np.round(clf.feature_importances_, 2)

# print(sorted(list(score), reverse=True))
print("Cummulative Importance Score: \n", np.cumsum(sorted(list(score), reverse=True)))
print(Counter(score))
features = []
for i in range(len(score)):
    if score[i] >= 0.02:
        features.append(X.columns[i])
        # print(df.columns[i])
print(features)
print(len(features))


In [ ]:
# utah_ssd_merged = pd.read_csv(filenames.get('utah_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
df_2 = emulab_hdd_merged.copy() # wisconsin_hdd_ssd_merged.copy()#utah_ssd_merged.copy()# wisconsin_ssd_merged.copy() #utah_ssd_merged_1g.copy() #wisconsin_ssd_merged.copy() #
remove_labels = [ 91, 100, 46]
df_2 = remove_labels_in_df(df_2, remove_labels)
df_2 = normalize_df(df_2)
df_2 = grouped_to_5_cat(df_2,161)
# df_2 = wisconsin_ssd_merged.copy() #utah_ssd_merged.copy()#
y_test = df_2.label_value
X_test = df_2[features] #.drop(columns="label_value")##[features] #.drop(columns="label_value")
# X_train, X_test, y_train, y_test = train_test_split(X,y)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))

# y_pred_5cat = change_labels_5(y_pred, 21)#change_labels_21(y_pred, 161)
# y_test_5cat = change_labels_5(y_test, 21)#change_labels_21(y_test, 161)
# print(np.round(metrics.accuracy_score(y_test_5cat, y_pred_5cat) * 100, 2))

In [ ]:
def change_labels_5(y, total_possible_labels):
    aggregated_labels = {}
    for i in range(total_possible_labels + 1):
        aggregated_labels[i] = i
    # 0 is normal and is one group itself
    # # group read together
    for i in range(1, 55):
        aggregated_labels.update({i: 1})

    # group write together
    for i in range(55, 109):
        aggregated_labels.update({i: 55})
    # group networks together
    for i in range(109, 145):
        aggregated_labels.update({i: 109})
    # group tcp configs together
    for i in range(145, 157):
        aggregated_labels.update({i: 145})
    # group with networks together
    for i in range(157, 161):
        aggregated_labels.update({i: 109})
    y_new = np.array([aggregated_labels[int(i)] for i in y])
    return y_new

def change_labels_21(y, total_possible_labels):
    aggregated_labels = {}
    for i in range(total_possible_labels + 1):
        aggregated_labels[i] = i
    # 0 is normal and is one group itself
    # group read_congestion_by_sender_sender_ost read levels together
    for i in range(1, 10):
        aggregated_labels.update({i: 1})
    # group read_congestion_by_clients_on_sender_ost read levels together
    for i in range(10, 19):
        aggregated_labels.update({i: 10})
    # group read_congestion_by_receiver_on_receiver_ost read levels together
    for i in range(19, 28):
        aggregated_labels.update({i: 19})
    # group read_congestion_by_clients_on_receiver_ost read levels together
    for i in range(28, 37):
        aggregated_labels.update({i: 28})
    # group read_congestion_by_sender_on_other_ost read levels together
    for i in range(37, 46):
        aggregated_labels.update({i: 37})
    # group read_congestion_by_writer_on_other_ost read levels together
    for i in range(46, 55):
        aggregated_labels.update({i: 46})
    # group write_congestion_by_sender_on_sender_ost read levels together
    for i in range(55, 64):
        aggregated_labels.update({i: 55})
    # group write_congestion_by_clients_on_sender_ost read levels together
    for i in range(64, 73):
        aggregated_labels.update({i: 64})
    # group write_congestion_by_receiver_on_receiver read levels together
    for i in range(73, 82):
        aggregated_labels.update({i: 73})
    # group write_congestion_by_clients_on_receiver read levels together
    for i in range(82, 91):
        aggregated_labels.update({i: 82})
    # group write_congestion_by_sender_on_other_ost read levels together
    for i in range(91, 100):
        aggregated_labels.update({i: 91})
    # group write_congestion_by_receiver_on_other_ost read levels together
    for i in range(100, 109):
        aggregated_labels.update({i: 100})
    # group network_anomaly_network_loss read levels together
    for i in range(109, 113):
        aggregated_labels.update({i: 109})
    # group network_anomaly_network_delay read levels together
    for i in range(113, 117):
        aggregated_labels.update({i: 113})
    # group network_anomaly_network_duplicate read levels together
    for i in range(117, 129):
        aggregated_labels.update({i: 117})
    # group network_anomaly_network_corrupt read levels together
    for i in range(129, 133):
        aggregated_labels.update({i: 129})
    # group network_anomaly_network_reorder read levels together
    for i in range(133, 145):
        aggregated_labels.update({i: 133})
    # group sys_config_tcp_send read levels together
    for i in range(145, 151):
        aggregated_labels.update({i: 145})
    # group sys_config_tcp_receive_buffer read levels together
    for i in range(151, 157):
        aggregated_labels.update({i: 151})
    for i in range(157, 161):
        aggregated_labels.update({i: 157})
    y_new = np.array([aggregated_labels[int(i)] for i in y])
    return y_new

In [ ]:
score = np.round(clf.feature_importances_, 2)

# print(sorted(list(score), reverse=True))
print("Cummulative Importance Score: \n", np.cumsum(sorted(list(score), reverse=True)))
print(Counter(score))
features = []
for i in range(len(score)):
    if score[i] >= 0.02:
        features.append(X.columns[i])
        # print(df.columns[i])
print(features)
print(len(features))


In [ ]:
y_test = test_df.label_value
X_test = test_df[features] #.drop(columns="label_value")
print(X_test.shape)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))
# Counter(y_pred)
y_pred_5cat = change_labels_5(y_pred, 21)#change_labels_21(y_pred, 161)
y_test_5cat = change_labels_5(y_test, 21)#change_labels_21(y_test, 161)
print(np.round(metrics.accuracy_score(y_test_5cat, y_pred_5cat) * 100, 2))
# y_pred_5cat[y_pred_5cat[:]==55]
# y_pred_5cat
# Counter(y_pred_5cat)

In [ ]:
new_df = pd.read_csv('./montage_analysis/normal.csv').drop(['time_stamp', 'through_put'], axis=1)
# new_df = pd.read_csv('./montage_analysis/montage_5_instance.csv').drop(['time_stamp', 'through_put'], axis=1)
new_df = normalize_df(new_df)
y_test = new_df.label_value
X_test = new_df[features] #.drop(columns="label_value")#[features] #.drop(columns="label_value")
print(X_test.shape)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))
print(Counter(y_pred))
# print(y_test)

y_pred_5cat = change_labels_5(y_pred, 21)#change_labels_21(y_pred, 161)
y_test_5cat = change_labels_5(y_test, 21)#change_labels(y_test, 161)
print(np.round(metrics.accuracy_score(y_test_5cat, y_pred_5cat) * 100, 2))
print(Counter(y_pred_5cat))
# Counter(y_pred_5cat)
# # new_df.loc[new_df['label_value']==0]
# np.round(X_test.mean(), 2)
# np.round(train_df[train_df.label_value == 0][features].std(), 2) / np.round(X_test.std(), 2)
# train_df.groupby("label_value").receiver_avg_waittime_md.mean()

In [ ]:
emulab_hdd_merged


In [ ]:
df = utah_ssd_merged_1g.copy()
df = normalize_df(df)
df = grouped_to_5_cat(df,161)
# df[df.label_value==0][features]
# df[features]
df[df.label_value==1][features].describe()

In [ ]:
df = utah_ssd_merged.copy()
# df.sender_send_buffer_value = df.sender_send_buffer_value/df.sender_avg_rtt_value
df.groupby('label_value').sender_send_buffer_value.mean()
# df = normalize_df(df)
# df = grouped_to_5_cat(df,161)
# df[(df.label_value==1) & (df.sender_cwnd_rate <= 1105.5) & (df.sender_retrans <= 1.5)
#    & (df.sender_req_active <= 565.5) & (df.sender_avg_send_value <= 0.1)][features]
# df[df.label_value==1][features].describe()
# df[df.sender_write_bytes>0].groupby('label_value').mean()

In [ ]:
df = utah_ssd_merged_1g.copy()
# df.sender_send_buffer_value = df.sender_send_buffer_value/df.sender_avg_rtt_value
df.groupby('label_value').sender_send_buffer_value.mean()

In [ ]:
df = utah_ssd_merged.copy()
df = normalize_df(df)
df = grouped_to_5_cat(df,161)
# df[(df.label_value==1) & (df.sender_cwnd_rate <= 1105.5) & (df.sender_retrans <= 1.5)
#    & (df.sender_req_active <= 565.5) & (df.sender_avg_send_value <= 0.1)][features]
df[df.label_value==1][features].describe()

In [ ]:
df = wisconsin_ssd_merged.copy()
df = normalize_df(df)
df = grouped_to_5_cat(df,161)
df[df.label_value==0][features]

In [ ]:
df[df.label_value==0][features]

In [ ]:
new_df[new_df.receiver_write_bytes <= 310378496.].receiver_write_bytes
# new_df[new_df.sender_cwnd_rate <= 330.].sender_cwnd_rate

In [ ]:
train_df[train_df.label_value == 0][features]

In [ ]:
# df = emulab_hdd_merged.copy()
# df = wisconsin_ssd_merged.copy()
# df = wisconsin_ssd_unmerged.copy()
df = utah_ssd_merged_1g.copy()
remove_labels = [ 91, 100, 46] #[]#[ 91, 100, 46]
df = remove_labels_in_df(df, remove_labels)
df = normalize_df(df)
df = grouped_to_5_cat(df,161)

train_df = df
# print(train_df.shape)

y = train_df.label_value
X = train_df[features]
X_train, X_test, y_train, y_test = train_test_split(X,y)
print(X_train.shape)
# X_train, y_train = RandomUnderSampler(sampling_strategy="all").fit_resample(X_train, y_train)
X_train, y_train = RandomOverSampler(sampling_strategy="all").fit_resample(X_train, y_train)

print(X_train.shape)
# clf = RandomForestClassifier(n_estimators=100)
clf = tree.DecisionTreeClassifier(max_depth=10)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(np.round(metrics.accuracy_score(y_test, y_pred) * 100, 2))


In [ ]:
class_names = [str(lbl) for lbl in sorted(Counter(train_df.label_value).keys())]
class_names

In [ ]:
# fn=['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
# cn=['setosa', 'versicolor', 'virginica']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (100,25), dpi=10)
tree.plot_tree(clf, feature_names = features,
               class_names=class_names,
               filled = True,
               fontsize=12);
fig.savefig('tree_10.pdf')

# tree.plot_tree(clf, feature_names=features, fontsize=12)

In [ ]:
df_2 = emulab_hdd_merged.copy()
remove_labels = [ 91, 100, 46]
df_2 = remove_labels_in_df(df_2, remove_labels)
df_2 = normalize_df(df_2)
df_2 = grouped_to_5_cat(df_2,161)
target_label = 145
df_2 = df_2[df_2.label_value == target_label]

# y_test = df_2.label_value.loc[2:3]
# X_test = df_2[features].loc[2:3,:]
y_test = df_2.label_value
X_test = df_2[features]
y_pred = clf.predict(X_test)
print(np.round(metrics.f1_score(y_test, y_pred, average='weighted') * 100, 2))

In [ ]:


decision_path = clf.decision_path(X_test.iloc[np.where(y_pred!=target_label)])
# print(decision_path)

fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (100,25), dpi=10)
class_names = [str(lbl) for lbl in sorted(Counter(train_df.label_value).keys())]
model_tree = tree.plot_tree(clf, feature_names = features,
               class_names=class_names,
               filled = True,
               fontsize=12);

for i in range(0,len(model_tree)):
     if i not in decision_path.indices:
         plt.setp(model_tree[i],visible=False)
fig.savefig('path_10.pdf')
plt.show()


In [ ]:
tmp_df = X_test.iloc[np.where(y_pred!=target_label)]
tmp_df
# tmp_df[tmp_df.sender_remote_ost_write_bytes <= 77594624.0]
# tmp_df[tmp_df.receiver_remote_ost_write_bytes <= 2.141]
# tmp_df[(tmp_df.receiver_remote_ost_write_bytes <= 2.141) & (tmp_df.sender_retrans <=0)]
tmp_df[ (tmp_df.sender_read_bytes_io > 182759424.0)&
       (tmp_df.receiver_remote_ost_write_bytes <= 2.141)
       & (tmp_df.sender_retrans >0) & (tmp_df.sender_req_active <= 569.5)]
#        (tmp_df.sender_send_buffer_value <= 459016.0)]
# tmp_df[tmp_df.receiver_write_bytes_io <= 112316416.0]

In [ ]:
np.where(y_pred!=target_label)
X_test.iloc[np.where(y_pred!=target_label)]
# X_test

In [ ]:
# utah_ssd_merged.loc[utah_ssd_merged['label_value']==0]


In [ ]:
def grouped_to_5_cat(df, total_possible_labels):
    aggregated_labels = {}
    for i in range(total_possible_labels + 1):
        aggregated_labels[i] = i
    # 0 is normal and is one group itself
    # # group read together
    for i in range(1, 55):
        aggregated_labels.update({i: 1})

    # group write together
    for i in range(55, 109):
        aggregated_labels.update({i: 55})
    # group networks together
    for i in range(109, 145):
        aggregated_labels.update({i: 109})
    # group tcp configs together
    for i in range(145, 157):
        aggregated_labels.update({i: 145})
    # group with networks together
    for i in range(157, 161):
        aggregated_labels.update({i: 109})
    y = [aggregated_labels[int(i)] for i in df[df.columns[len(df.columns) - 1]].values]
    df["label_value"] = y
    return df

In [ ]:
df = pd.read_csv(filenames.get('wisconsin_ssd_unmerged')).drop(['time_stamp', 'through_put'], axis=1)
# df = grouped_to_5_cat(df,161)
df.shape
Counter(df.label_value)

In [ ]:
t = pd.read_csv(filenames.get('utah_ssd_merged')).drop(['time_stamp', 'through_put'], axis=1)
t[t.label_value == 0].receiver_avg_waittime_md

In [ ]:
utah_ssd_unmerged_1g = pd.read_csv(filenames.get('emulab_hdd_unmerged')) #  wisconsin_ssd_unmerged
utah_ssd_unmerged_1g.groupby('label_value').through_put.mean()
# utah_ssd_unmerged_1g[utah_ssd_unmerged_1g.label_value==130].through_put